In [ ]:
# https://drive.google.com/file/d/1MCmntA3BOquC7BWxLcYDItohDgTPRKx8/view?usp=sharing
import gdown
file_id = '1MCmntA3BOquC7BWxLcYDItohDgTPRKx8'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'Crop_Data_Final.csv'
gdown.download(url, output, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1MCmntA3BOquC7BWxLcYDItohDgTPRKx8
To: /content/Crop_Data_Final.csv
100%|██████████| 214k/214k [00:00<00:00, 14.4MB/s]


'Crop_Data_Final.csv'

In [ ]:
import pandas as pd

# Read data from a CSV file
data = pd.read_csv('/content/Crop_Data_Final.csv')
print(data.head())


   Year   Dist Name  RICE AREA (1000 ha)  RICE PRODUCTION (1000 tons)  \
0  1966  Ahmednagar                   11                            5   
1  1967  Ahmednagar                    9                            8   
2  1968  Ahmednagar                    8                            7   
3  1969  Ahmednagar                    8                            5   
4  1970  Ahmednagar                    7                            6   

   RICE YIELD (Kg per ha)  WHEAT AREA (1000 ha)  WHEAT PRODUCTION (1000 tons)  \
0                     455                    38                            17   
1                     889                    40                            14   
2                     886                    43                            28   
3                     667                    49                            23   
4                     789                    47                            29   

   WHEAT YIELD (Kg per ha)  SORGHUM AREA (1000 ha)  \
0                   

#@ svr = SVR(kernel='rbf')

In [ ]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
file_path = "/content/Crop_Data_Final.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# List of crops and their yield attributes
crops = {
    "Rice": "RICE YIELD (Kg per ha)",
    "Wheat": "WHEAT YIELD (Kg per ha)",
    "Sorghum": "SORGHUM YIELD (Kg per ha)",
    "Pearl Millet": "PEARL MILLET YIELD (Kg per ha)",
    "Maize": "MAIZE YIELD (Kg per ha)",
    "Chickpea": "CHICKPEA YIELD (Kg per ha)",
    "Pigeonpea": "PIGEONPEA YIELD (Kg per ha)",
    "Groundnut": "GROUNDNUT YIELD (Kg per ha)",
    "Sesamum": "SESAMUM YIELD (Kg per ha)",
    "Oilseeds": "OILSEEDS YIELD (Kg per ha)",
    "Sugarcane": "SUGARCANE YIELD (Kg per ha)",
    "Cotton": "COTTON YIELD (Kg per ha)"
}

# Iterate through each district
districts = data["Dist Name"].unique()
results = {}

for district in districts:
    district_data = data[data["Dist Name"] == district]

    print(f"Processing district: {district}")
    district_results = {}

    for crop, yield_column in crops.items():
        if yield_column not in district_data.columns:
            print(f"Skipping crop: {crop} (Yield data not available)")
            continue

        # Prepare input features (X) and target (y)
        X = district_data.drop(columns=["Year", "Dist Name", yield_column])
        y = district_data[yield_column]

        # Check if there's enough data to run SVR
        if len(X) < 10 or y.isnull().any():
            print(f"Not enough data for crop: {crop} in district: {district}")
            continue

        # Handle missing values
        X = X.fillna(0)
        y = y.fillna(0)

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Standardize the features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Fit SVR model
        svr = SVR(kernel='rbf')
        svr.fit(X_train_scaled, y_train)

        # Make predictions
        y_pred = svr.predict(X_test_scaled)

        # Evaluate the model
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print(f"Crop: {crop}, MSE: {mse:.2f}, R2: {r2:.2f}")
        district_results[crop] = {"MSE": mse, "R2": r2}

    results[district] = district_results

# Save results to a file
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.to_csv("district_crop_svr_results.csv")
print("Results saved to 'district_crop_svr_results.csv'")


Processing district: Ahmednagar
Crop: Rice, MSE: 165762.32, R2: -0.02
Crop: Wheat, MSE: 112994.57, R2: -0.01
Crop: Sorghum, MSE: 16288.84, R2: -0.06
Crop: Pearl Millet, MSE: 51594.44, R2: -0.55
Crop: Maize, MSE: 298820.32, R2: -0.54
Crop: Chickpea, MSE: 18554.35, R2: -0.11
Crop: Pigeonpea, MSE: 15137.78, R2: 0.00
Crop: Groundnut, MSE: 172493.00, R2: -0.13
Crop: Sesamum, MSE: 3659.80, R2: -0.02
Crop: Oilseeds, MSE: 62742.56, R2: -0.30
Crop: Sugarcane, MSE: 1053677.21, R2: -0.02
Crop: Cotton, MSE: 8409.76, R2: -0.01
Processing district: Akola
Crop: Rice, MSE: 216713.35, R2: -0.28
Crop: Wheat, MSE: 383864.41, R2: -0.16
Crop: Sorghum, MSE: 374374.50, R2: -0.73
Crop: Pearl Millet, MSE: 37215.25, R2: -1.33
Crop: Maize, MSE: 422261.02, R2: -0.05
Crop: Chickpea, MSE: 133588.04, R2: -0.24
Crop: Pigeonpea, MSE: 63486.74, R2: -0.54
Crop: Groundnut, MSE: 297534.32, R2: -0.06
Crop: Sesamum, MSE: 6598.24, R2: -0.69
Crop: Oilseeds, MSE: 248319.70, R2: -0.35
Crop: Sugarcane, MSE: 6693020.53, R2: -0.01

svr = SVR(kernel='poly')

In [ ]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
file_path = "/content/Crop_Data_Final.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# List of crops and their yield attributes
crops = {
    "Rice": "RICE YIELD (Kg per ha)",
    "Wheat": "WHEAT YIELD (Kg per ha)",
    "Sorghum": "SORGHUM YIELD (Kg per ha)",
    "Pearl Millet": "PEARL MILLET YIELD (Kg per ha)",
    "Maize": "MAIZE YIELD (Kg per ha)",
    "Chickpea": "CHICKPEA YIELD (Kg per ha)",
    "Pigeonpea": "PIGEONPEA YIELD (Kg per ha)",
    "Groundnut": "GROUNDNUT YIELD (Kg per ha)",
    "Sesamum": "SESAMUM YIELD (Kg per ha)",
    "Oilseeds": "OILSEEDS YIELD (Kg per ha)",
    "Sugarcane": "SUGARCANE YIELD (Kg per ha)",
    "Cotton": "COTTON YIELD (Kg per ha)"
}

# Iterate through each district
districts = data["Dist Name"].unique()
results = {}

for district in districts:
    district_data = data[data["Dist Name"] == district]

    print(f"Processing district: {district}")
    district_results = {}

    for crop, yield_column in crops.items():
        if yield_column not in district_data.columns:
            print(f"Skipping crop: {crop} (Yield data not available)")
            continue

        # Prepare input features (X) and target (y)
        X = district_data.drop(columns=["Year", "Dist Name", yield_column])
        y = district_data[yield_column]

        # Check if there's enough data to run SVR
        if len(X) < 10 or y.isnull().any():
            print(f"Not enough data for crop: {crop} in district: {district}")
            continue

        # Handle missing values
        X = X.fillna(0)
        y = y.fillna(0)

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Standardize the features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Fit SVR model
        svr = SVR(kernel='poly')
        svr.fit(X_train_scaled, y_train)

        # Make predictions
        y_pred = svr.predict(X_test_scaled)

        # Evaluate the model
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print(f"Crop: {crop}, MSE: {mse:.2f}, R2: {r2:.2f}")
        district_results[crop] = {"MSE": mse, "R2": r2}

    results[district] = district_results

# Save results to a file
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.to_csv("district_crop_svr_results.csv")
print("Results saved to 'district_crop_svr_results.csv'")


Processing district: Ahmednagar
Crop: Rice, MSE: 165777.31, R2: -0.02
Crop: Wheat, MSE: 111310.66, R2: 0.01
Crop: Sorghum, MSE: 15580.75, R2: -0.01
Crop: Pearl Millet, MSE: 50912.42, R2: -0.52
Crop: Maize, MSE: 294953.96, R2: -0.52
Crop: Chickpea, MSE: 17581.90, R2: -0.05
Crop: Pigeonpea, MSE: 15078.39, R2: 0.00
Crop: Groundnut, MSE: 175320.71, R2: -0.14
Crop: Sesamum, MSE: 3872.37, R2: -0.08
Crop: Oilseeds, MSE: 62260.88, R2: -0.29
Crop: Sugarcane, MSE: 1055692.64, R2: -0.03
Crop: Cotton, MSE: 8315.12, R2: 0.00
Processing district: Akola
Crop: Rice, MSE: 217812.44, R2: -0.29
Crop: Wheat, MSE: 375542.28, R2: -0.13
Crop: Sorghum, MSE: 374984.98, R2: -0.73
Crop: Pearl Millet, MSE: 35523.26, R2: -1.22
Crop: Maize, MSE: 418987.84, R2: -0.04
Crop: Chickpea, MSE: 128752.20, R2: -0.19
Crop: Pigeonpea, MSE: 62125.24, R2: -0.51
Crop: Groundnut, MSE: 287937.48, R2: -0.02
Crop: Sesamum, MSE: 6599.54, R2: -0.69
Crop: Oilseeds, MSE: 243811.73, R2: -0.33
Crop: Sugarcane, MSE: 6693716.25, R2: -0.01
C

fine tuned kernel=rbf

In [ ]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
file_path = "/content/Crop_Data_Final.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# List of crops and their yield attributes
# crops = {
#     "Rice": "RICE YIELD (Kg per ha)",
#     "Wheat": "WHEAT YIELD (Kg per ha)",
#     "Sorghum": "SORGHUM YIELD (Kg per ha)",
#     "Pearl Millet": "PEARL MILLET YIELD (Kg per ha)",
#     "Maize": "MAIZE YIELD (Kg per ha)",
#     "Chickpea": "CHICKPEA YIELD (Kg per ha)",
#     "Pigeonpea": "PIGEONPEA YIELD (Kg per ha)",
#     "Groundnut": "GROUNDNUT YIELD (Kg per ha)",
#     "Sesamum": "SESAMUM YIELD (Kg per ha)",
#     "Oilseeds": "OILSEEDS YIELD (Kg per ha)",
#     "Sugarcane": "SUGARCANE YIELD (Kg per ha)",
#     "Cotton": "COTTON YIELD (Kg per ha)"
# }


crops = [
    ('RICE YIELD (Kg per ha)', 'RICE PRODUCTION (1000 tons)'),
    ('WHEAT YIELD (Kg per ha)', 'WHEAT PRODUCTION (1000 tons)'),
    ('SORGHUM YIELD (Kg per ha)', 'SORGHUM PRODUCTION (1000 tons)'),
    ('PEARL MILLET YIELD (Kg per ha)', 'PEARL MILLET PRODUCTION (1000 tons)'),
    ('MAIZE YIELD (Kg per ha)', 'MAIZE PRODUCTION (1000 tons)'),
    ('CHICKPEA YIELD (Kg per ha)', 'CHICKPEA PRODUCTION (1000 tons)'),
    ('PIGEONPEA YIELD (Kg per ha)', 'PIGEONPEA PRODUCTION (1000 tons)'),
    ('MINOR PULSES YIELD (Kg per ha)', 'MINOR PULSES PRODUCTION (1000 tons)'),
    ('GROUNDNUT YIELD (Kg per ha)', 'GROUNDNUT PRODUCTION (1000 tons)'),
    ('SESAMUM YIELD (Kg per ha)', 'SESAMUM PRODUCTION (1000 tons)'),
    ('OILSEEDS YIELD (Kg per ha)', 'OILSEEDS PRODUCTION (1000 tons)'),
    ('SUGARCANE YIELD (Kg per ha)', 'SUGARCANE PRODUCTION (1000 tons)'),
    ('COTTON YIELD (Kg per ha)', 'COTTON PRODUCTION (1000 tons)'),
    # Add any additional crops if required
]

# Set up parameter grid for SVR
param_grid = {
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 0.5, 1],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1],
    'kernel': ['rbf', 'poly', 'linear']
}

# Iterate through each district
districts = data["Dist Name"].unique()
results = {}

for district in districts:
    district_data = data[data["Dist Name"] == district]

    print(f"Processing district: {district}")
    district_results = {}

    for crop, yield_column in crops.items():
        if yield_column not in district_data.columns:
            print(f"Skipping crop: {crop} (Yield data not available)")
            continue

        # Prepare input features (X) and target (y)
        X = district_data.drop(columns=["Year", "Dist Name", yield_column])
        y = district_data[yield_column]

        # Handle missing values
        X = X.fillna(0)
        y = y.fillna(0)

        # Normalize target variable
        scaler_y = MinMaxScaler()
        y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

        # Standardize features
        scaler_X = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(X_train)
        X_test_scaled = scaler_X.transform(X_test)

        # Grid search for best hyperparameters
        grid_search = GridSearchCV(SVR(), param_grid, cv=3, scoring='r2', n_jobs=-1)
        grid_search.fit(X_train_scaled, y_train.ravel())

        best_params = grid_search.best_params_
        print(f"Best parameters for {crop} in {district}: {best_params}")

        # Train final model with best parameters
        svr = SVR(**best_params)
        svr.fit(X_train_scaled, y_train.ravel())

        # Predict and evaluate
        y_pred_scaled = svr.predict(X_test_scaled)
        y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
        y_test = scaler_y.inverse_transform(y_test)

        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print(f"Crop: {crop}, MSE: {mse:.2f}, R2: {r2:.2f}")
        district_results[crop] = {"MSE": mse, "R2": r2, "Best Params": best_params}

    results[district] = district_results

# Save results to a file
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.to_csv("district_crop_svr_results.csv")
print("Results saved to 'district_crop_svr_results.csv'")


Processing district: Ahmednagar
Best parameters for Rice in Ahmednagar: {'C': 10, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'poly'}
Crop: Rice, MSE: 107391.92, R2: 0.34
Best parameters for Wheat in Ahmednagar: {'C': 1, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
Crop: Wheat, MSE: 46857.84, R2: 0.58
Best parameters for Sorghum in Ahmednagar: {'C': 0.1, 'epsilon': 0.01, 'gamma': 'scale', 'kernel': 'linear'}
Crop: Sorghum, MSE: 954.54, R2: 0.94
Best parameters for Pearl Millet in Ahmednagar: {'C': 0.1, 'epsilon': 0.01, 'gamma': 'scale', 'kernel': 'linear'}
Crop: Pearl Millet, MSE: 3334.28, R2: 0.90
Best parameters for Maize in Ahmednagar: {'C': 1, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
Crop: Maize, MSE: 110232.90, R2: 0.43
Best parameters for Chickpea in Ahmednagar: {'C': 0.1, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Crop: Chickpea, MSE: 3694.90, R2: 0.78
Best parameters for Pigeonpea in Ahmednagar: {'C': 0.1, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'linear

In [ ]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
file_path = "/content/Crop_Data_Final.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# List of targets for prediction (Yield, Production, and Area for each crop)
targets = {
    "Rice": {
        "Yield": "RICE YIELD (Kg per ha)",
        "Production": "RICE PRODUCTION (1000 tons)",
        "Area": "RICE AREA (1000 ha)"
    },
    "Wheat": {
        "Yield": "WHEAT YIELD (Kg per ha)",
        "Production": "WHEAT PRODUCTION (1000 tons)",
        "Area": "WHEAT AREA (1000 ha)"
    },
    "Sorghum": {
        "Yield": "SORGHUM YIELD (Kg per ha)",
        "Production": "SORGHUM PRODUCTION (1000 tons)",
        "Area": "SORGHUM AREA (1000 ha)"
    },
    "Pearl Millet": {
        "Yield": "PEARL MILLET YIELD (Kg per ha)",
        "Production": "PEARL MILLET PRODUCTION (1000 tons)",
        "Area": "PEARL MILLET AREA (1000 ha)"
    },
    "Maize": {
        "Yield": "MAIZE YIELD (Kg per ha)",
        "Production": "MAIZE PRODUCTION (1000 tons)",
        "Area": "MAIZE AREA (1000 ha)"
    },
    "Chickpea": {
        "Yield": "CHICKPEA YIELD (Kg per ha)",
        "Production": "CHICKPEA PRODUCTION (1000 tons)",
        "Area": "CHICKPEA AREA (1000 ha)"
    },
    "Pigeonpea": {
        "Yield": "PIGEONPEA YIELD (Kg per ha)",
        "Production": "PIGEONPEA PRODUCTION (1000 tons)",
        "Area": "PIGEONPEA AREA (1000 ha)"
    },
    "Groundnut": {
        "Yield": "GROUNDNUT YIELD (Kg per ha)",
        "Production": "GROUNDNUT PRODUCTION (1000 tons)",
        "Area": "GROUNDNUT AREA (1000 ha)"
    },
    "Sesamum": {
        "Yield": "SESAMUM YIELD (Kg per ha)",
        "Production": "SESAMUM PRODUCTION (1000 tons)",
        "Area": "SESAMUM AREA (1000 ha)"
    },
    "Oilseeds": {
        "Yield": "OILSEEDS YIELD (Kg per ha)",
        "Production": "OILSEEDS PRODUCTION (1000 tons)",
        "Area": "OILSEEDS AREA (1000 ha)"
    },
    "Sugarcane": {
        "Yield": "SUGARCANE YIELD (Kg per ha)",
        "Production": "SUGARCANE PRODUCTION (1000 tons)",
        "Area": "SUGARCANE AREA (1000 ha)"
    },
    "Cotton": {
        "Yield": "COTTON YIELD (Kg per ha)",
        "Production": "COTTON PRODUCTION (1000 tons)",
        "Area": "COTTON AREA (1000 ha)"
    },
    # Add more crops here if needed
}

# Set up parameter grid for SVR
param_grid = {
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 0.5, 1],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1],
    'kernel': ['rbf', 'poly', 'linear']
}

# Iterate through each district
districts = data["Dist Name"].unique()
results = {}

for district in districts:
    district_data = data[data["Dist Name"] == district]

    print(f"Processing district: {district}")
    district_results = {}

    for crop, variables in targets.items():
        crop_results = {}

        for variable_name, column in variables.items():
            if column not in district_data.columns:
                print(f"Skipping {variable_name} for crop: {crop} (Data not available)")
                continue

            # Prepare input features (X) and target (y)
            X = district_data.drop(columns=["Year", "Dist Name"] + list(variables.values()))
            y = district_data[column]

            # Handle missing values
            X = X.fillna(0)
            y = y.fillna(0)

            # Normalize target variable
            scaler_y = MinMaxScaler()
            y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

            # Train-test split
            X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

            # Standardize features
            scaler_X = StandardScaler()
            X_train_scaled = scaler_X.fit_transform(X_train)
            X_test_scaled = scaler_X.transform(X_test)

            # Grid search for best hyperparameters
            grid_search = GridSearchCV(SVR(), param_grid, cv=3, scoring='r2', n_jobs=-1)
            grid_search.fit(X_train_scaled, y_train.ravel())

            best_params = grid_search.best_params_
            print(f"Best parameters for {variable_name} of {crop} in {district}: {best_params}")

            # Train final model with best parameters
            svr = SVR(**best_params)
            svr.fit(X_train_scaled, y_train.ravel())

            # Predict and evaluate
            y_pred_scaled = svr.predict(X_test_scaled)
            y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
            y_test = scaler_y.inverse_transform(y_test)

            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)

            print(f"{variable_name} for {crop}: MSE = {mse:.2f}, R2 = {r2:.2f}")
            crop_results[variable_name] = {
                "MSE": mse,
                "R2": r2,
                "Best Params": best_params
            }

        district_results[crop] = crop_results

    results[district] = district_results

# Save results to a file
results_df = pd.json_normalize(results, sep='_').T  # Flatten the nested dictionary
results_df.columns = ["Metric"]
results_df.to_csv("district_crop_predictions_results.csv")
print("Results saved to 'district_crop_predictions_results.csv'")


Processing district: Ahmednagar
Best parameters for Yield of Rice in Ahmednagar: {'C': 1, 'epsilon': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}
Yield for Rice: MSE = 122006.40, R2 = 0.25
Best parameters for Production of Rice in Ahmednagar: {'C': 1, 'epsilon': 0.01, 'gamma': 0.1, 'kernel': 'rbf'}
Production for Rice: MSE = 11.40, R2 = -0.01
Best parameters for Area of Rice in Ahmednagar: {'C': 10, 'epsilon': 0.01, 'gamma': 'scale', 'kernel': 'rbf'}
Area for Rice: MSE = 4.09, R2 = -2.18
Best parameters for Yield of Wheat in Ahmednagar: {'C': 1, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
Yield for Wheat: MSE = 53641.98, R2 = 0.52
Best parameters for Production of Wheat in Ahmednagar: {'C': 0.1, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Production for Wheat: MSE = 563.95, R2 = 0.87
Best parameters for Area of Wheat in Ahmednagar: {'C': 10, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
Area for Wheat: MSE = 85.84, R2 = 0.92
Best parameters for Yield of Sorghum in Ahmednaga

KeyboardInterrupt: 

In [ ]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Load the dataset
file_path = "/content/Crop_Data_Final.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# List of targets for prediction (Yield, Production, and Area for each crop)
targets = {
    "Rice": {
        "Yield": "RICE YIELD (Kg per ha)",
        "Production": "RICE PRODUCTION (1000 tons)",
        "Area": "RICE AREA (1000 ha)"
    },
    "Wheat": {
        "Yield": "WHEAT YIELD (Kg per ha)",
        "Production": "WHEAT PRODUCTION (1000 tons)",
        "Area": "WHEAT AREA (1000 ha)"
    },
    "Sorghum": {
        "Yield": "SORGHUM YIELD (Kg per ha)",
        "Production": "SORGHUM PRODUCTION (1000 tons)",
        "Area": "SORGHUM AREA (1000 ha)"
    },
    "Pearl Millet": {
        "Yield": "PEARL MILLET YIELD (Kg per ha)",
        "Production": "PEARL MILLET PRODUCTION (1000 tons)",
        "Area": "PEARL MILLET AREA (1000 ha)"
    },
    "Maize": {
        "Yield": "MAIZE YIELD (Kg per ha)",
        "Production": "MAIZE PRODUCTION (1000 tons)",
        "Area": "MAIZE AREA (1000 ha)"
    },
    "Chickpea": {
        "Yield": "CHICKPEA YIELD (Kg per ha)",
        "Production": "CHICKPEA PRODUCTION (1000 tons)",
        "Area": "CHICKPEA AREA (1000 ha)"
    },
    "Pigeonpea": {
        "Yield": "PIGEONPEA YIELD (Kg per ha)",
        "Production": "PIGEONPEA PRODUCTION (1000 tons)",
        "Area": "PIGEONPEA AREA (1000 ha)"
    },
    "Groundnut": {
        "Yield": "GROUNDNUT YIELD (Kg per ha)",
        "Production": "GROUNDNUT PRODUCTION (1000 tons)",
        "Area": "GROUNDNUT AREA (1000 ha)"
    },
    "Sesamum": {
        "Yield": "SESAMUM YIELD (Kg per ha)",
        "Production": "SESAMUM PRODUCTION (1000 tons)",
        "Area": "SESAMUM AREA (1000 ha)"
    },
    "Oilseeds": {
        "Yield": "OILSEEDS YIELD (Kg per ha)",
        "Production": "OILSEEDS PRODUCTION (1000 tons)",
        "Area": "OILSEEDS AREA (1000 ha)"
    },
    "Sugarcane": {
        "Yield": "SUGARCANE YIELD (Kg per ha)",
        "Production": "SUGARCANE PRODUCTION (1000 tons)",
        "Area": "SUGARCANE AREA (1000 ha)"
    },
    "Cotton": {
        "Yield": "COTTON YIELD (Kg per ha)",
        "Production": "COTTON PRODUCTION (1000 tons)",
        "Area": "COTTON AREA (1000 ha)"
    },
    # Add more crops here if needed
}

# Future years to predict
future_years = list(range(2018, 2046))

# Set up parameter grid for SVR
param_grid = {
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 0.5, 1],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1],
    'kernel': ['rbf', 'poly', 'linear']
}

# Iterate through each district
districts = data["Dist Name"].unique()
results = {}
future_predictions = []

for district in districts:
    district_data = data[data["Dist Name"] == district]

    print(f"Processing district: {district}")
    district_results = {}

    for crop, variables in targets.items():
        crop_results = {}

        for variable_name, column in variables.items():
            if column not in district_data.columns:
                print(f"Skipping {variable_name} for crop: {crop} (Data not available)")
                continue

            # Prepare input features (X) and target (y)
            X = district_data.drop(columns=["Year", "Dist Name"] + list(variables.values()))
            y = district_data[column]

            # Handle missing values
            X = X.fillna(0)
            y = y.fillna(0)

            # Normalize target variable
            scaler_y = MinMaxScaler()
            y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

            # Train-test split
            X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

            # Standardize features
            scaler_X = StandardScaler()
            X_train_scaled = scaler_X.fit_transform(X_train)
            X_test_scaled = scaler_X.transform(X_test)

            # Grid search for best hyperparameters
            grid_search = GridSearchCV(SVR(), param_grid, cv=3, scoring='r2', n_jobs=-1)
            grid_search.fit(X_train_scaled, y_train.ravel())

            best_params = grid_search.best_params_
            print(f"Best parameters for {variable_name} of {crop} in {district}: {best_params}")

            # Train final model with best parameters
            svr = SVR(**best_params)
            svr.fit(X_train_scaled, y_train.ravel())

            # Predict and evaluate
            y_pred_scaled = svr.predict(X_test_scaled)
            y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
            y_test = scaler_y.inverse_transform(y_test)

            mse = mean_squared_error(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            mape = (abs((y_test - y_pred) / y_test).mean()) * 100
            r2 = r2_score(y_test, y_pred)

            print(f"{variable_name} for {crop}: MSE = {mse:.2f}, MAE = {mae:.2f}, MAPE = {mape:.2f}%, R2 = {r2:.2f}")
            crop_results[variable_name] = {
                "MAE": mae,
                "MAPE": mape,
                "MSE": mse,
                "R2": r2,
                "Best Params": best_params
            }

            # Predict for future years
            future_features = pd.DataFrame({
                "Year": future_years,
                **{col: district_data[col].mean() for col in X.columns}  # Use mean or default values for other features
            })
            future_features_scaled = scaler_X.transform(future_features.drop(columns=["Year"]))
            future_pred_scaled = svr.predict(future_features_scaled)
            future_pred = scaler_y.inverse_transform(future_pred_scaled.reshape(-1, 1))

            # Store future predictions
            for year, pred in zip(future_years, future_pred):
                future_predictions.append({
                    "District": district,
                    "Crop": crop,
                    "Variable": variable_name,
                    "Year": year,
                    "Predicted Value": pred[0]
                })

        district_results[crop] = crop_results

    results[district] = district_results

# Save results to a file
results_df = pd.json_normalize(results, sep='_').T  # Flatten the nested dictionary
results_df.columns = ["Metric"]
results_df.to_csv("district_crop_predictions_results.csv")
print("Results saved to 'district_crop_predictions_results.csv'")

# Save future predictions
future_df = pd.DataFrame(future_predictions)
future_df.to_csv("future_predictions.csv", index=False)
print("Future predictions saved to 'future_predictions.csv'")


Processing district: Ahmednagar
Best parameters for Yield of Rice in Ahmednagar: {'C': 1, 'epsilon': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}
Yield for Rice: MSE = 122006.40, MAE = 283.12, MAPE = 41.18%, R2 = 0.25
Best parameters for Production of Rice in Ahmednagar: {'C': 1, 'epsilon': 0.01, 'gamma': 0.1, 'kernel': 'rbf'}
Production for Rice: MSE = 11.40, MAE = 2.89, MAPE = 49.67%, R2 = -0.01
Best parameters for Area of Rice in Ahmednagar: {'C': 10, 'epsilon': 0.01, 'gamma': 'scale', 'kernel': 'rbf'}
Area for Rice: MSE = 4.09, MAE = 1.63, MAPE = 20.69%, R2 = -2.18
Best parameters for Yield of Wheat in Ahmednagar: {'C': 1, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
Yield for Wheat: MSE = 53641.98, MAE = 211.05, MAPE = 21.39%, R2 = 0.52
Best parameters for Production of Wheat in Ahmednagar: {'C': 0.1, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Production for Wheat: MSE = 563.95, MAE = 18.61, MAPE = 23.14%, R2 = 0.87
Best parameters for Area of Wheat in Ahmednagar: {'C': 1

<ipython-input-3-aee4b4a59768>:144: RuntimeWarning: divide by zero encountered in divide
  mape = (abs((y_test - y_pred) / y_test).mean()) * 100


Best parameters for Area of Sesamum in Ahmednagar: {'C': 0.1, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
Area for Sesamum: MSE = 0.65, MAE = 0.69, MAPE = inf%, R2 = 0.62


<ipython-input-3-aee4b4a59768>:144: RuntimeWarning: divide by zero encountered in divide
  mape = (abs((y_test - y_pred) / y_test).mean()) * 100


Best parameters for Yield of Oilseeds in Ahmednagar: {'C': 0.1, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Yield for Oilseeds: MSE = 27696.38, MAE = 144.51, MAPE = 33.31%, R2 = 0.43
Best parameters for Production of Oilseeds in Ahmednagar: {'C': 0.1, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Production for Oilseeds: MSE = 478.24, MAE = 18.06, MAPE = 46.61%, R2 = 0.39
Best parameters for Area of Oilseeds in Ahmednagar: {'C': 0.1, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
Area for Oilseeds: MSE = 332.38, MAE = 14.52, MAPE = 14.47%, R2 = 0.24
Best parameters for Yield of Sugarcane in Ahmednagar: {'C': 1, 'epsilon': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}
Yield for Sugarcane: MSE = 724881.87, MAE = 696.78, MAPE = 8.38%, R2 = 0.29
Best parameters for Production of Sugarcane in Ahmednagar: {'C': 0.1, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Production for Sugarcane: MSE = 22829.13, MAE = 100.42, MAPE = 17.93%, R2 = -0.42
Best parameters for Area of S

...

Best parameters for Area of Maize in Yeotmal: {'C': 1, 'epsilon': 0.01, 'gamma': 'scale', 'kernel': 'poly'}
Area for Maize: MSE = 0.19, MAE = 0.24, MAPE = inf%, R2 = -0.28


<ipython-input-3-aee4b4a59768>:144: RuntimeWarning: divide by zero encountered in divide
  mape = (abs((y_test - y_pred) / y_test).mean()) * 100


Best parameters for Yield of Chickpea in Yeotmal: {'C': 1, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
Yield for Chickpea: MSE = 12430.53, MAE = 96.67, MAPE = 18.16%, R2 = 0.79
Best parameters for Production of Chickpea in Yeotmal: {'C': 1, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
Production for Chickpea: MSE = 34.78, MAE = 3.79, MAPE = 38.63%, R2 = 0.88
Best parameters for Area of Chickpea in Yeotmal: {'C': 1, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'poly'}
Area for Chickpea: MSE = 64.07, MAE = 4.84, MAPE = 20.39%, R2 = 0.75
Best parameters for Yield of Pigeonpea in Yeotmal: {'C': 1, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'poly'}
Yield for Pigeonpea: MSE = 19521.08, MAE = 129.01, MAPE = 17.16%, R2 = 0.28
Best parameters for Production of Pigeonpea in Yeotmal: {'C': 0.1, 'epsilon': 0.01, 'gamma': 'scale', 'kernel': 'rbf'}
Production for Pigeonpea: MSE = 255.29, MAE = 12.95, MAPE = 19.91%, R2 = 0.76
Best parameters for Area of Pigeonpea in Yeotmal: {'C': 1, 'epsilon': 

<ipython-input-3-aee4b4a59768>:144: RuntimeWarning: divide by zero encountered in divide
  mape = (abs((y_test - y_pred) / y_test).mean()) * 100


Best parameters for Area of Sesamum in Yeotmal: {'C': 1, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
Area for Sesamum: MSE = 1.15, MAE = 0.89, MAPE = 22.47%, R2 = 0.93
Best parameters for Yield of Oilseeds in Yeotmal: {'C': 1, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
Yield for Oilseeds: MSE = 108878.66, MAE = 255.03, MAPE = 31.25%, R2 = 0.57
Best parameters for Production of Oilseeds in Yeotmal: {'C': 1, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
Production for Oilseeds: MSE = 2532.90, MAE = 33.99, MAPE = 29.14%, R2 = 0.84
Best parameters for Area of Oilseeds in Yeotmal: {'C': 1, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
Area for Oilseeds: MSE = 1384.32, MAE = 24.17, MAPE = 28.28%, R2 = 0.87
Best parameters for Yield of Sugarcane in Yeotmal: {'C': 1, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'poly'}
Yield for Sugarcane: MSE = 428856.77, MAE = 528.93, MAPE = 7.18%, R2 = -0.23
Best parameters for Production of Sugarcane in Yeotmal: {'C': 1, 'epsilon': 0.01, 'gam

**svr again**-not this

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Function to preprocess data, fine-tune SVR, and run for one target variable
def run_svr_for_target(district_data, target_column, forecast_years):
    # Ensure data is sorted by year
    district_data.sort_values('Year', inplace=True)

    # Define features (X) and target (y)
    X = district_data[['Year']]
    y = district_data[target_column]

    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

    # Define SVR and parameter grid for fine-tuning
    svr = SVR()
    param_grid = {
        'kernel': ['rbf'],  # Using RBF kernel for SVR
        'C': [1, 10, 100, 1000],
        'gamma': ['scale', 0.1, 0.01, 0.001],
        'epsilon': [0.1, 0.2, 0.5]
    }

    # Fine-tune using GridSearchCV
    grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Best model from GridSearchCV
    best_svr = grid_search.best_estimator_

    # Make predictions on the test set
    y_pred = best_svr.predict(X_test)

    # Evaluate the model
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100  # Mean Absolute Percentage Error
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    # Forecast future values
    future_years = pd.DataFrame({'Year': range(X['Year'].max() + 1, X['Year'].max() + forecast_years + 1)})
    future_predictions = best_svr.predict(future_years)

    return mae, mape, rmse, r2, future_years['Year'].tolist(), future_predictions.tolist(), grid_search.best_params_

# Function to run SVR for all districts and target columns
def run_svr_for_all(data, target_columns, forecast_years, output_csv_path):
    districts = data['Dist Name'].unique()
    results = []

    for district in districts:
        print(f"Processing District: {district}")
        district_data = data[data['Dist Name'] == district].copy()

        for target_column in target_columns:
            print(f"  Target Variable: {target_column}")
            mae, mape, rmse, r2, future_years, future_predictions, best_params = run_svr_for_target(
                district_data, target_column, forecast_years
            )

            # Append results
            for year, prediction in zip(future_years, future_predictions):
                results.append({
                    'District': district,
                    'Target': target_column,
                    'MAE': mae,
                    'MAPE': mape,
                    'RMSE': rmse,
                    'R²': r2,
                    'Year': year,
                    'Prediction': prediction,
                    'Best Params': best_params
                })

    # Save results to CSV
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_csv_path, index=False)
    print(f"Results saved to {output_csv_path}")

# Load your dataset
data_path = "/content/Crop_Data_Final.csv"  # Replace with the actual file path
data = pd.read_csv(data_path)

# List of target columns
target_columns = [
    "RICE AREA (1000 ha)", "RICE PRODUCTION (1000 tons)", "RICE YIELD (Kg per ha)",
    "WHEAT AREA (1000 ha)", "WHEAT PRODUCTION (1000 tons)", "WHEAT YIELD (Kg per ha)",
    "SORGHUM AREA (1000 ha)", "SORGHUM PRODUCTION (1000 tons)", "SORGHUM YIELD (Kg per ha)",
    "PEARL MILLET AREA (1000 ha)", "PEARL MILLET PRODUCTION (1000 tons)", "PEARL MILLET YIELD (Kg per ha)",
    "MAIZE AREA (1000 ha)", "MAIZE PRODUCTION (1000 tons)", "MAIZE YIELD (Kg per ha)",
    "CHICKPEA AREA (1000 ha)", "CHICKPEA PRODUCTION (1000 tons)", "CHICKPEA YIELD (Kg per ha)",
    "PIGEONPEA AREA (1000 ha)", "PIGEONPEA PRODUCTION (1000 tons)", "PIGEONPEA YIELD (Kg per ha)",
    "MINOR PULSES AREA (1000 ha)", "MINOR PULSES PRODUCTION (1000 tons)", "MINOR PULSES YIELD (Kg per ha)",
    "GROUNDNUT AREA (1000 ha)", "GROUNDNUT PRODUCTION (1000 tons)", "GROUNDNUT YIELD (Kg per ha)",
    "SESAMUM AREA (1000 ha)", "SESAMUM PRODUCTION (1000 tons)", "SESAMUM YIELD (Kg per ha)",
    "OILSEEDS AREA (1000 ha)", "OILSEEDS PRODUCTION (1000 tons)", "OILSEEDS YIELD (Kg per ha)",
    "SUGARCANE AREA (1000 ha)", "SUGARCANE PRODUCTION (1000 tons)", "SUGARCANE YIELD (Kg per ha)",
    "COTTON AREA (1000 ha)", "COTTON PRODUCTION (1000 tons)", "COTTON YIELD (Kg per ha)",
    "FRUITS AND VEGETABLES AREA (1000 ha)",
    "NITROGEN SHARE IN NPK (Percent)", "NITROGEN PER HA OF NCA (Kg per ha)", "NITROGEN PER HA OF GCA (Kg per ha)",
    "PHOSPHATE CONSUMPTION (tons)", "PHOSPHATE SHARE IN NPK (Percent)",
    "PHOSPHATE PER HA OF NCA (Kg per ha)", "PHOSPHATE PER HA OF GCA (Kg per ha)",
    "POTASH CONSUMPTION (tons)", "POTASH SHARE IN NPK (Percent)",
    "POTASH PER HA OF NCA (Kg per ha)", "POTASH PER HA OF GCA (Kg per ha)",
    "Min Temp (Centigrate)", "Max Temp (Centigrate)", "Precipitation (mm)",
    "Irrigated Area", "Annual Rainfall"
]

# Forecast years and output path
forecast_years = 29  # Forecast from 2017 to 2045
output_csv_path = "/content/svr_forecast_results.csv"  # Replace with desired output path

# Run the SVR for all districts and target columns
run_svr_for_all(data, target_columns, forecast_years, output_csv_path)


Processing District: Ahmednagar
  Target Variable: RICE AREA (1000 ha)
  Target Variable: RICE PRODUCTION (1000 tons)
  Target Variable: RICE YIELD (Kg per ha)
  Target Variable: WHEAT AREA (1000 ha)
  Target Variable: WHEAT PRODUCTION (1000 tons)
  Target Variable: WHEAT YIELD (Kg per ha)
  Target Variable: SORGHUM AREA (1000 ha)
  Target Variable: SORGHUM PRODUCTION (1000 tons)
  Target Variable: SORGHUM YIELD (Kg per ha)
  Target Variable: PEARL MILLET AREA (1000 ha)
  Target Variable: PEARL MILLET PRODUCTION (1000 tons)
  Target Variable: PEARL MILLET YIELD (Kg per ha)
  Target Variable: MAIZE AREA (1000 ha)
  Target Variable: MAIZE PRODUCTION (1000 tons)
  Target Variable: MAIZE YIELD (Kg per ha)
  Target Variable: CHICKPEA AREA (1000 ha)
  Target Variable: CHICKPEA PRODUCTION (1000 tons)
  Target Variable: CHICKPEA YIELD (Kg per ha)
  Target Variable: PIGEONPEA AREA (1000 ha)
  Target Variable: PIGEONPEA PRODUCTION (1000 tons)
  Target Variable: PIGEONPEA YIELD (Kg per ha)
  Targ

this one svr

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from joblib import Parallel, delayed

# Function to preprocess data, fine-tune SVR, and run for one target variable
def run_svr_for_target(district_data, target_column, forecast_years):
    district_data.sort_values('Year', inplace=True)

    X = district_data[['Year']]
    y = district_data[target_column]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

    svr = SVR()
    param_grid = {
        'kernel': ['rbf'],
        'C': [10, 100],
        'gamma': ['scale', 0.1],
        'epsilon': [0.1, 0.2]
    }

    grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, n_jobs=1)
    grid_search.fit(X_train, y_train)

    best_svr = grid_search.best_estimator_
    y_pred = best_svr.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    future_years = pd.DataFrame({'Year': range(X['Year'].max() + 1, X['Year'].max() + forecast_years + 1)})
    future_predictions = best_svr.predict(future_years)

    return mae, mape, rmse, r2, future_years['Year'].tolist(), future_predictions.tolist(), grid_search.best_params_

# Function to process each district using threading
def process_district_threaded(district, data, target_columns, forecast_years):
    district_data = data[data['Dist Name'] == district].copy()
    results = []
    for target_column in target_columns:
        mae, mape, rmse, r2, future_years, future_predictions, best_params = run_svr_for_target(
            district_data, target_column, forecast_years
        )
        for year, prediction in zip(future_years, future_predictions):
            results.append({
                'District': district,
                'Target': target_column,
                'MAE': mae,
                'MAPE': mape,
                'RMSE': rmse,
                'R²': r2,
                'Year': year,
                'Prediction': prediction,
                'Best Params': best_params
            })
    return results

# Function to run SVR for all districts and target columns using threading
def run_svr_for_all_parallel(data, target_columns, forecast_years, output_csv_path):
    districts = data['Dist Name'].unique()

    # Use threading for parallel processing
    results = Parallel(n_jobs=-1, backend='threading')(
        delayed(process_district_threaded)(district, data, target_columns, forecast_years) for district in districts
    )

    # Combine results and save to CSV
    results_flat = [item for sublist in results for item in sublist]
    results_df = pd.DataFrame(results_flat)
    results_df.to_csv(output_csv_path, index=False)
    print(f"Results saved to {output_csv_path}")

# Load your dataset
data_path = "/content/Crop_Data_Final.csv"  # Replace with the actual file path
data = pd.read_csv(data_path)

# List of target columns
target_columns = [
    "RICE AREA (1000 ha)", "RICE PRODUCTION (1000 tons)", "RICE YIELD (Kg per ha)",
    "WHEAT AREA (1000 ha)", "WHEAT PRODUCTION (1000 tons)", "WHEAT YIELD (Kg per ha)",
    "SORGHUM AREA (1000 ha)", "SORGHUM PRODUCTION (1000 tons)", "SORGHUM YIELD (Kg per ha)",
    "PEARL MILLET AREA (1000 ha)", "PEARL MILLET PRODUCTION (1000 tons)", "PEARL MILLET YIELD (Kg per ha)",
    "MAIZE AREA (1000 ha)", "MAIZE PRODUCTION (1000 tons)", "MAIZE YIELD (Kg per ha)",
    "CHICKPEA AREA (1000 ha)", "CHICKPEA PRODUCTION (1000 tons)", "CHICKPEA YIELD (Kg per ha)",
    "PIGEONPEA AREA (1000 ha)", "PIGEONPEA PRODUCTION (1000 tons)", "PIGEONPEA YIELD (Kg per ha)",
    "MINOR PULSES AREA (1000 ha)", "MINOR PULSES PRODUCTION (1000 tons)", "MINOR PULSES YIELD (Kg per ha)",
    "GROUNDNUT AREA (1000 ha)", "GROUNDNUT PRODUCTION (1000 tons)", "GROUNDNUT YIELD (Kg per ha)",
    "SESAMUM AREA (1000 ha)", "SESAMUM PRODUCTION (1000 tons)", "SESAMUM YIELD (Kg per ha)",
    "OILSEEDS AREA (1000 ha)", "OILSEEDS PRODUCTION (1000 tons)", "OILSEEDS YIELD (Kg per ha)",
    "SUGARCANE AREA (1000 ha)", "SUGARCANE PRODUCTION (1000 tons)", "SUGARCANE YIELD (Kg per ha)",
    "COTTON AREA (1000 ha)", "COTTON PRODUCTION (1000 tons)", "COTTON YIELD (Kg per ha)",
    "FRUITS AND VEGETABLES AREA (1000 ha)",
    "NITROGEN SHARE IN NPK (Percent)", "NITROGEN PER HA OF NCA (Kg per ha)", "NITROGEN PER HA OF GCA (Kg per ha)",
    "PHOSPHATE CONSUMPTION (tons)", "PHOSPHATE SHARE IN NPK (Percent)",
    "PHOSPHATE PER HA OF NCA (Kg per ha)", "PHOSPHATE PER HA OF GCA (Kg per ha)",
    "POTASH CONSUMPTION (tons)", "POTASH SHARE IN NPK (Percent)",
    "POTASH PER HA OF NCA (Kg per ha)", "POTASH PER HA OF GCA (Kg per ha)",
    "Min Temp (Centigrate)", "Max Temp (Centigrate)", "Precipitation (mm)",
    "Irrigated Area", "Annual Rainfall"
]

# Forecast years and output path
forecast_years = 29  # Forecast from 2017 to 2045
output_csv_path = "/content/svr_forecast_results.csv"  # Replace with desired output path

# Run the SVR for all districts and target columns
run_svr_for_all_parallel(data, target_columns, forecast_years, output_csv_path)


Results saved to /content/svr_forecast_results.csv


# svr running

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Ensure output directory exists
output_dir = "district_predictions"
os.makedirs(output_dir, exist_ok=True)

# Load dataset
file_path = "/content/Crop_Data_Final.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# List of all attributes (excluding 'Year' and 'Dist Name')
attributes = [
    "RICE AREA (1000 ha)", "RICE PRODUCTION (1000 tons)", "RICE YIELD (Kg per ha)",
    "WHEAT AREA (1000 ha)", "WHEAT PRODUCTION (1000 tons)", "WHEAT YIELD (Kg per ha)",
    "SORGHUM AREA (1000 ha)", "SORGHUM PRODUCTION (1000 tons)", "SORGHUM YIELD (Kg per ha)",
    "PEARL MILLET AREA (1000 ha)", "PEARL MILLET PRODUCTION (1000 tons)", "PEARL MILLET YIELD (Kg per ha)",
    "MAIZE AREA (1000 ha)", "MAIZE PRODUCTION (1000 tons)", "MAIZE YIELD (Kg per ha)",
    "CHICKPEA AREA (1000 ha)", "CHICKPEA PRODUCTION (1000 tons)", "CHICKPEA YIELD (Kg per ha)",
    "PIGEONPEA AREA (1000 ha)", "PIGEONPEA PRODUCTION (1000 tons)", "PIGEONPEA YIELD (Kg per ha)",
    "MINOR PULSES AREA (1000 ha)", "MINOR PULSES PRODUCTION (1000 tons)", "MINOR PULSES YIELD (Kg per ha)",
    "GROUNDNUT AREA (1000 ha)", "GROUNDNUT PRODUCTION (1000 tons)", "GROUNDNUT YIELD (Kg per ha)",
    "SESAMUM AREA (1000 ha)", "SESAMUM PRODUCTION (1000 tons)", "SESAMUM YIELD (Kg per ha)",
    "OILSEEDS AREA (1000 ha)", "OILSEEDS PRODUCTION (1000 tons)", "OILSEEDS YIELD (Kg per ha)",
    "SUGARCANE AREA (1000 ha)", "SUGARCANE PRODUCTION (1000 tons)", "SUGARCANE YIELD (Kg per ha)",
    "COTTON AREA (1000 ha)", "COTTON PRODUCTION (1000 tons)", "COTTON YIELD (Kg per ha)",
    "FRUITS AND VEGETABLES AREA (1000 ha)", "NITROGEN SHARE IN NPK (Percent)",
    "NITROGEN PER HA OF NCA (Kg per ha)", "NITROGEN PER HA OF GCA (Kg per ha)",
    "PHOSPHATE CONSUMPTION (tons)", "PHOSPHATE SHARE IN NPK (Percent)",
    "PHOSPHATE PER HA OF NCA (Kg per ha)", "PHOSPHATE PER HA OF GCA (Kg per ha)",
    "POTASH CONSUMPTION (tons)", "POTASH SHARE IN NPK (Percent)",
    "POTASH PER HA OF NCA (Kg per ha)", "POTASH PER HA OF GCA (Kg per ha)",
    "Min Temp (Centigrate)", "Max Temp (Centigrate)", "Precipitation (mm)",
    "Irrigated Area", "Annual Rainfall"
]

# Define future years for prediction
future_years = list(range(2018, 2046))

# Parameter grid for Random Forest
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 15, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt']
}

# Initialize results
future_predictions = []

# Iterate through each district
for district in data["Dist Name"].unique():
    district_data = data[data["Dist Name"] == district].copy()
    print(f"Processing district: {district}")

    for attribute in attributes:
        if attribute not in district_data.columns:
            print(f"Skipping attribute {attribute} for district {district} (Data not available)")
            continue

        # Prepare input features and target
        X = district_data.drop(columns=["Year", "Dist Name", attribute], errors='ignore')
        y = district_data[attribute]
        X.fillna(0, inplace=True)
        y.fillna(0, inplace=True)

        # Normalize target
        scaler_y = MinMaxScaler()
        y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

        # Standardize features
        scaler_X = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(X_train)
        X_test_scaled = scaler_X.transform(X_test)

        # Grid search for best parameters
        grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, scoring='r2', n_jobs=-1)
        grid_search.fit(X_train_scaled, y_train.ravel())

        best_params = grid_search.best_params_
        print(f"Best parameters for {attribute} in {district}: {best_params}")

        # Train final model
        rf = RandomForestRegressor(**best_params, random_state=42)
        rf.fit(X_train_scaled, y_train.ravel())
        y_pred_scaled = rf.predict(X_test_scaled)

        # Clip predictions to ensure within range
        y_pred_scaled = np.clip(y_pred_scaled, 0, 1)

        # Inverse transform predictions
        y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
        y_test = scaler_y.inverse_transform(y_test)

        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        mape = (np.abs((y_test - y_pred) / y_test).mean()) * 100
        r2 = r2_score(y_test, y_pred)

        print(f"{attribute}: MSE={mse:.2f}, MAE={mae:.2f}, MAPE={mape:.2f}%, R2={r2:.2f}")

        # Predict future values
        future_features = pd.DataFrame({
            "Year": future_years,
            **{col: district_data[col].mean() for col in X.columns}
        })
        future_features_scaled = scaler_X.transform(future_features.drop(columns=["Year"]))
        future_pred_scaled = rf.predict(future_features_scaled)

        # Clip and inverse transform future predictions
        future_pred_scaled = np.clip(future_pred_scaled, 0, 1)
        future_pred = scaler_y.inverse_transform(future_pred_scaled.reshape(-1, 1))

        for year, pred in zip(future_years, future_pred):
            future_predictions.append({
                "District": district,
                "Attribute": attribute,
                "Year": year,
                "Predicted Value": pred[0]
            })

# Save all future predictions to a single CSV
future_df = pd.DataFrame(future_predictions)
future_df.to_csv("future_predictions.csv", index=False)
print("All future predictions saved to 'future_predictions.csv'")


Processing district: Ahmednagar
Best parameters for RICE AREA (1000 ha) in Ahmednagar: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 150}
RICE AREA (1000 ha): MSE=1.89, MAE=1.05, MAPE=13.31%, R2=-0.47
Best parameters for RICE PRODUCTION (1000 tons) in Ahmednagar: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 150}
RICE PRODUCTION (1000 tons): MSE=7.48, MAE=2.15, MAPE=36.97%, R2=0.33
Best parameters for RICE YIELD (Kg per ha) in Ahmednagar: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 50}
RICE YIELD (Kg per ha): MSE=92544.14, MAE=239.45, MAPE=33.04%, R2=0.43
Best parameters for WHEAT AREA (1000 ha) in Ahmednagar: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}
WHEAT AREA (1000 ha): MSE=341.01, MAE=13.45, MAPE=13.76%, R2=0.68
Best parameters for WHEAT P

<ipython-input-7-42ce7ec3dee1>:106: RuntimeWarning: divide by zero encountered in divide
  mape = (np.abs((y_test - y_pred) / y_test).mean()) * 100


Best parameters for SESAMUM PRODUCTION (1000 tons) in Ahmednagar: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
SESAMUM PRODUCTION (1000 tons): MSE=0.51, MAE=0.59, MAPE=inf%, R2=-1.06


<ipython-input-7-42ce7ec3dee1>:106: RuntimeWarning: divide by zero encountered in divide
  mape = (np.abs((y_test - y_pred) / y_test).mean()) * 100


Best parameters for SESAMUM YIELD (Kg per ha) in Ahmednagar: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 150}
SESAMUM YIELD (Kg per ha): MSE=2631.63, MAE=43.15, MAPE=19.19%, R2=0.27
Best parameters for OILSEEDS AREA (1000 ha) in Ahmednagar: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 150}
OILSEEDS AREA (1000 ha): MSE=298.83, MAE=12.57, MAPE=12.26%, R2=0.32
Best parameters for OILSEEDS PRODUCTION (1000 tons) in Ahmednagar: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
OILSEEDS PRODUCTION (1000 tons): MSE=283.02, MAE=14.43, MAPE=26.49%, R2=0.64
Best parameters for OILSEEDS YIELD (Kg per ha) in Ahmednagar: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 150}
OILSEEDS YIELD (Kg per ha): MSE=5870.92, MAE=57.33, MAPE=9.77%, R2=0.88
Best parameters for S

...

Best parameters for SORGHUM PRODUCTION (1000 tons) in Bhandara: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
SORGHUM PRODUCTION (1000 tons): MSE=12.48, MAE=1.93, MAPE=inf%, R2=0.76


<ipython-input-7-42ce7ec3dee1>:106: RuntimeWarning: divide by zero encountered in divide
  mape = (np.abs((y_test - y_pred) / y_test).mean()) * 100


Best parameters for SORGHUM YIELD (Kg per ha) in Bhandara: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
SORGHUM YIELD (Kg per ha): MSE=81110.57, MAE=158.47, MAPE=inf%, R2=-0.10


<ipython-input-7-42ce7ec3dee1>:106: RuntimeWarning: divide by zero encountered in divide
  mape = (np.abs((y_test - y_pred) / y_test).mean()) * 100


KeyboardInterrupt: 

Random forest regressor

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load the dataset
file_path = "/content/Crop_Data_Final.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# Define target variables for each crop
targets = {
    "Rice": {
        "Yield": "RICE YIELD (Kg per ha)",
        "Production": "RICE PRODUCTION (1000 tons)",
        "Area": "RICE AREA (1000 ha)"
    },
    "Wheat": {
        "Yield": "WHEAT YIELD (Kg per ha)",
        "Production": "WHEAT PRODUCTION (1000 tons)",
        "Area": "WHEAT AREA (1000 ha)"
    },
    "Sorghum": {
        "Yield": "SORGHUM YIELD (Kg per ha)",
        "Production": "SORGHUM PRODUCTION (1000 tons)",
        "Area": "SORGHUM AREA (1000 ha)"
    },
    "Pearl Millet": {
        "Yield": "PEARL MILLET YIELD (Kg per ha)",
        "Production": "PEARL MILLET PRODUCTION (1000 tons)",
        "Area": "PEARL MILLET AREA (1000 ha)"
    },
    "Maize": {
        "Yield": "MAIZE YIELD (Kg per ha)",
        "Production": "MAIZE PRODUCTION (1000 tons)",
        "Area": "MAIZE AREA (1000 ha)"
    },
    "Chickpea": {
        "Yield": "CHICKPEA YIELD (Kg per ha)",
        "Production": "CHICKPEA PRODUCTION (1000 tons)",
        "Area": "CHICKPEA AREA (1000 ha)"
    },
    "Pigeonpea": {
        "Yield": "PIGEONPEA YIELD (Kg per ha)",
        "Production": "PIGEONPEA PRODUCTION (1000 tons)",
        "Area": "PIGEONPEA AREA (1000 ha)"
    },
    "Groundnut": {
        "Yield": "GROUNDNUT YIELD (Kg per ha)",
        "Production": "GROUNDNUT PRODUCTION (1000 tons)",
        "Area": "GROUNDNUT AREA (1000 ha)"
    },
    "Sesamum": {
        "Yield": "SESAMUM YIELD (Kg per ha)",
        "Production": "SESAMUM PRODUCTION (1000 tons)",
        "Area": "SESAMUM AREA (1000 ha)"
    },
    "Oilseeds": {
        "Yield": "OILSEEDS YIELD (Kg per ha)",
        "Production": "OILSEEDS PRODUCTION (1000 tons)",
        "Area": "OILSEEDS AREA (1000 ha)"
    },
    "Sugarcane": {
        "Yield": "SUGARCANE YIELD (Kg per ha)",
        "Production": "SUGARCANE PRODUCTION (1000 tons)",
        "Area": "SUGARCANE AREA (1000 ha)"
    },
    "Cotton": {
        "Yield": "COTTON YIELD (Kg per ha)",
        "Production": "COTTON PRODUCTION (1000 tons)",
        "Area": "COTTON AREA (1000 ha)"
    },
    # Add more crops here if needed
}

# Define future years for prediction
future_years = list(range(2018, 2046))

# Parameter grid for Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize results containers
results = {}
future_predictions = []

# Iterate through each district
for district in data["Dist Name"].unique():
    district_data = data[data["Dist Name"] == district]
    print(f"Processing district: {district}")
    district_results = {}

    for crop, variables in targets.items():
        crop_results = {}
        for variable_name, column in variables.items():
            if column not in district_data.columns:
                print(f"Skipping {variable_name} for crop {crop} in {district} (Data not available)")
                continue

            # Prepare input features and target
            X = district_data.drop(columns=["Year", "Dist Name"] + list(variables.values()), errors='ignore')
            y = district_data[column]
            X.fillna(0, inplace=True)
            y.fillna(0, inplace=True)

            # Normalize target
            scaler_y = MinMaxScaler()
            y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

            # Train-test split
            X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

            # Standardize features
            scaler_X = StandardScaler()
            X_train_scaled = scaler_X.fit_transform(X_train)
            X_test_scaled = scaler_X.transform(X_test)

            # Grid search for best parameters
            grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=3, scoring='r2', n_jobs=-1)
            grid_search.fit(X_train_scaled, y_train.ravel())

            best_params = grid_search.best_params_
            print(f"Best parameters for {variable_name} of {crop} in {district}: {best_params}")

            # Train and evaluate final model
            rf = RandomForestRegressor(**best_params)
            rf.fit(X_train_scaled, y_train.ravel())
            y_pred_scaled = rf.predict(X_test_scaled)
            y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
            y_test = scaler_y.inverse_transform(y_test)

            mse = mean_squared_error(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            mape = (abs((y_test - y_pred) / y_test).mean()) * 100
            r2 = r2_score(y_test, y_pred)

            print(f"{variable_name} for {crop}: MSE={mse:.2f}, MAE={mae:.2f}, MAPE={mape:.2f}%, R2={r2:.2f}")
            crop_results[variable_name] = {
                "MSE": mse,
                "MAE": mae,
                "MAPE": mape,
                "R2": r2,
                "Best Params": best_params
            }

            # Predict for future years
            future_features = pd.DataFrame({
                "Year": future_years,
                **{col: district_data[col].mean() for col in X.columns}
            })
            future_features_scaled = scaler_X.transform(future_features.drop(columns=["Year"]))
            future_pred_scaled = rf.predict(future_features_scaled)
            future_pred = scaler_y.inverse_transform(future_pred_scaled.reshape(-1, 1))

            for year, pred in zip(future_years, future_pred):
                future_predictions.append({
                    "District": district,
                    "Crop": crop,
                    "Variable": variable_name,
                    "Year": year,
                    "Predicted Value": pred[0]
                })

        district_results[crop] = crop_results

    results[district] = district_results

# Save results
results_df = pd.json_normalize(results, sep='_').T
results_df.columns = ["Metric"]
results_df.to_csv("district_crop_predictions_results.csv")
print("Results saved to 'district_crop_predictions_results.csv'")

future_df = pd.DataFrame(future_predictions)
future_df.to_csv("future_predictions.csv", index=False)
print("Future predictions saved to 'future_predictions.csv'")


Processing district: Ahmednagar


<ipython-input-2-801c73ff7d83>:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Load the dataset
file_path = "/content/Crop_Data_Final.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# Define target variables for each crop
targets = {
    "Rice": {
        "Yield": "RICE YIELD (Kg per ha)",
        "Production": "RICE PRODUCTION (1000 tons)",
        "Area": "RICE AREA (1000 ha)"
    },
    "Wheat": {
        "Yield": "WHEAT YIELD (Kg per ha)",
        "Production": "WHEAT PRODUCTION (1000 tons)",
        "Area": "WHEAT AREA (1000 ha)"
    },
    "Sorghum": {
        "Yield": "SORGHUM YIELD (Kg per ha)",
        "Production": "SORGHUM PRODUCTION (1000 tons)",
        "Area": "SORGHUM AREA (1000 ha)"
    },
    "Pearl Millet": {
        "Yield": "PEARL MILLET YIELD (Kg per ha)",
        "Production": "PEARL MILLET PRODUCTION (1000 tons)",
        "Area": "PEARL MILLET AREA (1000 ha)"
    },
    "Maize": {
        "Yield": "MAIZE YIELD (Kg per ha)",
        "Production": "MAIZE PRODUCTION (1000 tons)",
        "Area": "MAIZE AREA (1000 ha)"
    },
    "Chickpea": {
        "Yield": "CHICKPEA YIELD (Kg per ha)",
        "Production": "CHICKPEA PRODUCTION (1000 tons)",
        "Area": "CHICKPEA AREA (1000 ha)"
    },
    "Pigeonpea": {
        "Yield": "PIGEONPEA YIELD (Kg per ha)",
        "Production": "PIGEONPEA PRODUCTION (1000 tons)",
        "Area": "PIGEONPEA AREA (1000 ha)"
    },
    "Groundnut": {
        "Yield": "GROUNDNUT YIELD (Kg per ha)",
        "Production": "GROUNDNUT PRODUCTION (1000 tons)",
        "Area": "GROUNDNUT AREA (1000 ha)"
    },
    "Sesamum": {
        "Yield": "SESAMUM YIELD (Kg per ha)",
        "Production": "SESAMUM PRODUCTION (1000 tons)",
        "Area": "SESAMUM AREA (1000 ha)"
    },
    "Oilseeds": {
        "Yield": "OILSEEDS YIELD (Kg per ha)",
        "Production": "OILSEEDS PRODUCTION (1000 tons)",
        "Area": "OILSEEDS AREA (1000 ha)"
    },
    "Sugarcane": {
        "Yield": "SUGARCANE YIELD (Kg per ha)",
        "Production": "SUGARCANE PRODUCTION (1000 tons)",
        "Area": "SUGARCANE AREA (1000 ha)"
    },
    "Cotton": {
        "Yield": "COTTON YIELD (Kg per ha)",
        "Production": "COTTON PRODUCTION (1000 tons)",
        "Area": "COTTON AREA (1000 ha)"
    },
    # Add more crops here if needed
}

# Define future years for prediction
future_years = list(range(2018, 2046))

# Parameter grid for Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']  # Removed 'auto'
}


# Initialize results containers
results = {}
future_predictions = []

# Iterate through each district
for district in data["Dist Name"].unique():
    district_data = data[data["Dist Name"] == district]
    print(f"\nProcessing district: {district}")
    district_results = {}

    for crop, variables in targets.items():
        crop_results = {}
        for variable_name, column in variables.items():
            if column not in district_data.columns:
                print(f"  Skipping {variable_name} for crop {crop} in {district} (Data not available)")
                continue

            try:
                # Prepare input features and target
                X = district_data.drop(columns=["Year", "Dist Name"] + list(variables.values()), errors='ignore')
                y = district_data[column]

                # Handle missing values
                X.fillna(X.median(), inplace=True)
                y.fillna(y.median(), inplace=True)

                # Normalize target
                scaler_y = MinMaxScaler()
                y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

                # Train-test split
                X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

                # Standardize features
                scaler_X = StandardScaler()
                X_train_scaled = scaler_X.fit_transform(X_train)
                X_test_scaled = scaler_X.transform(X_test)

                try:
                      grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=3, scoring='r2', n_jobs=-1)
                      grid_search.fit(X_train_scaled, y_train.ravel())
                      best_params = grid_search.best_params_
                except Exception as e:
                  print(f"GridSearchCV failed: {e}")
                  best_params = {}  # Use a default parameter set as a fallback


                # Grid search for best parameters
                # grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=3, scoring='r2', n_jobs=-1)
                # grid_search.fit(X_train_scaled, y_train.ravel())

                # best_params = grid_search.best_params_
                print(f"    Best parameters for {variable_name} of {crop}: {best_params}")

                # Train and evaluate final model
                rf = RandomForestRegressor(**best_params)
                rf.fit(X_train_scaled, y_train.ravel())
                y_pred_scaled = rf.predict(X_test_scaled)
                y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
                y_test = scaler_y.inverse_transform(y_test)

                mse = mean_squared_error(y_test, y_pred)
                mae = mean_absolute_error(y_test, y_pred)
                mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-10))) * 100
                r2 = r2_score(y_test, y_pred)

                print(f"      {variable_name} for {crop}: MSE={mse:.2f}, MAE={mae:.2f}, MAPE={mape:.2f}%, R2={r2:.2f}")
                crop_results[variable_name] = {
                    "MSE": mse,
                    "MAE": mae,
                    "MAPE": mape,
                    "R2": r2,
                    "Best Params": best_params
                }

                # Predict for future years
                future_features = pd.DataFrame({
                    "Year": future_years,
                    **{col: district_data[col].mean() for col in X.columns}
                })
                future_features_scaled = scaler_X.transform(future_features.drop(columns=["Year"]))
                future_pred_scaled = rf.predict(future_features_scaled)
                future_pred = scaler_y.inverse_transform(future_pred_scaled.reshape(-1, 1))

                for year, pred in zip(future_years, future_pred):
                    future_predictions.append({
                        "District": district,
                        "Crop": crop,
                        "Variable": variable_name,
                        "Year": year,
                        "Predicted Value": pred[0]
                    })

            except Exception as e:
                print(f"      Error processing {variable_name} for {crop} in {district}: {e}")

        district_results[crop] = crop_results

    results[district] = district_results

# Save results
results_df = pd.json_normalize(results, sep='_').T
results_df.columns = ["Metric"]
results_df.to_csv("district_crop_predictions_results.csv")
print("\nResults saved to 'district_crop_predictions_results.csv'")

future_df = pd.DataFrame(future_predictions)
future_df.to_csv("future_predictions.csv", index=False)
print("Future predictions saved to 'future_predictions.csv'")



Processing district: Ahmednagar


<ipython-input-5-c4f13523b396>:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(y.median(), inplace=True)


    Best parameters for Yield of Rice: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50}
      Yield for Rice: MSE=102060.42, MAE=273.94, MAPE=37.91%, R2=0.37


<ipython-input-5-c4f13523b396>:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(y.median(), inplace=True)


    Best parameters for Production of Rice: {'max_depth': 30, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
      Production for Rice: MSE=10.54, MAE=2.63, MAPE=42.12%, R2=0.06


...

    Best parameters for Production of Maize: {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 50}
      Production for Maize: MSE=0.86, MAE=0.69, MAPE=239659782072.35%, R2=0.70


<ipython-input-5-c4f13523b396>:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(y.median(), inplace=True)


    Best parameters for Area of Maize: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 50}
      Area for Maize: MSE=0.64, MAE=0.47, MAPE=72113341227.50%, R2=0.55


<ipython-input-5-c4f13523b396>:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(y.median(), inplace=True)


KeyboardInterrupt: 

svr1

---

svr1_fine_tuning_results_svr

---
svr1_forecasted_values_svr


In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pandas as pd
import numpy as np

# Load the dataset
file_path = '/content/Crop_Data_Final.csv'
data = pd.read_csv(file_path)

# Future years to forecast
future_years = [2025, 2030, 2035, 2040, 2045, 2050]

# Crop-related target columns
crop_columns = {
    'Rice': ['RICE AREA (1000 ha)', 'RICE PRODUCTION (1000 tons)', 'RICE YIELD (Kg per ha)'],
    'Wheat': ['WHEAT AREA (1000 ha)', 'WHEAT PRODUCTION (1000 tons)', 'WHEAT YIELD (Kg per ha)'],
    'Sorghum': ['SORGHUM AREA (1000 ha)', 'SORGHUM PRODUCTION (1000 tons)', 'SORGHUM YIELD (Kg per ha)'],
    'Pearl Millet': ['PEARL MILLET AREA (1000 ha)', 'PEARL MILLET PRODUCTION (1000 tons)', 'PEARL MILLET YIELD (Kg per ha)'],
    'Maize': ['MAIZE AREA (1000 ha)', 'MAIZE PRODUCTION (1000 tons)', 'MAIZE YIELD (Kg per ha)'],
    'Chickpea': ['CHICKPEA AREA (1000 ha)', 'CHICKPEA PRODUCTION (1000 tons)', 'CHICKPEA YIELD (Kg per ha)'],
    'Pigeonpea': ['PIGEONPEA AREA (1000 ha)', 'PIGEONPEA PRODUCTION (1000 tons)', 'PIGEONPEA YIELD (Kg per ha)'],
    'Minor Pulses': ['MINOR PULSES AREA (1000 ha)', 'MINOR PULSES PRODUCTION (1000 tons)', 'MINOR PULSES YIELD (Kg per ha)'],
    'Groundnut': ['GROUNDNUT AREA (1000 ha)', 'GROUNDNUT PRODUCTION (1000 tons)', 'GROUNDNUT YIELD (Kg per ha)'],
    'Sesamum': ['SESAMUM AREA (1000 ha)', 'SESAMUM PRODUCTION (1000 tons)', 'SESAMUM YIELD (Kg per ha)'],
    'Oilseeds': ['OILSEEDS AREA (1000 ha)', 'OILSEEDS PRODUCTION (1000 tons)', 'OILSEEDS YIELD (Kg per ha)'],
    'Sugarcane': ['SUGARCANE AREA (1000 ha)', 'SUGARCANE PRODUCTION (1000 tons)', 'SUGARCANE YIELD (Kg per ha)'],
    'Cotton': ['COTTON AREA (1000 ha)', 'COTTON PRODUCTION (1000 tons)', 'COTTON YIELD (Kg per ha)'],
    'Fruits and Vegetables': ['FRUITS AND VEGETABLES AREA (1000 ha)'],
    'Fertilizers': ['NITROGEN SHARE IN NPK (Percent)', 'PHOSPHATE SHARE IN NPK (Percent)', 'POTASH SHARE IN NPK (Percent)'],
    'Soil Nutrients': ['NITROGEN PER HA OF NCA (Kg per ha)', 'NITROGEN PER HA OF GCA (Kg per ha)',
                       'PHOSPHATE PER HA OF NCA (Kg per ha)', 'PHOSPHATE PER HA OF GCA (Kg per ha)',
                       'POTASH PER HA OF NCA (Kg per ha)', 'POTASH PER HA OF GCA (Kg per ha)'],
    'Weather': ['Min Temp (Centigrate)', 'Max Temp (Centigrate)', 'Precipitation (mm)', 'Irrigated Area', 'Annual Rainfall']
}

# Hyperparameter tuning grid for SVR
param_distributions = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.1],
    'epsilon': [0.1, 0.01]
}

# Function to calculate metrics
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return mse, r2, mae, rmse

# Preprocessing function for scaling
def preprocess_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, scaler

# Function to process crops for a district
def process_crop(data, crop, target_column):
    X = data.drop(columns=['Year', 'Dist Name'] + crop_columns[crop], errors='ignore')
    y = data[target_column]

    if len(y) < 5:
        return None  # Skip if insufficient data

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Preprocess the data
    X_train_scaled, X_test_scaled, scaler = preprocess_data(X_train, X_test)

    # Hyperparameter tuning
    svr = SVR()
    random_search = RandomizedSearchCV(
        estimator=svr,
        param_distributions=param_distributions,
        n_iter=10,
        cv=2,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )
    random_search.fit(X_train_scaled, y_train)

    # Get the best model
    best_model = random_search.best_estimator_
    best_params = random_search.best_params_

    # Evaluate on test data
    y_pred = best_model.predict(X_test_scaled)
    mse, r2, mae, rmse = calculate_metrics(y_test, y_pred)

    # Prepare future data for forecasting
    future_data = data.iloc[:len(future_years)].copy()
    future_data['Year'] = future_years
    future_X = future_data.drop(columns=['Year', 'Dist Name'] + crop_columns[crop], errors='ignore')
    future_X_scaled = scaler.transform(future_X)
    future_predictions = best_model.predict(future_X_scaled)

    return {
        'fine_tuning': {
            'Crop': crop,
            'Target Column': target_column,
            'Best Parameters': best_params,
            'MSE': mse,
            'R²': r2,
            'MAE': mae,
            'RMSE': rmse
        },
        'forecasting': {
            'Future Years': future_years,
            'Predictions': future_predictions.tolist()
        }
    }

# Process each crop and district
districts = data['Dist Name'].unique()
results = []

for district in districts:
    district_data = data[data['Dist Name'] == district]
    for crop, columns in crop_columns.items():
        for target_column in columns:
            result = process_crop(district_data, crop, target_column)
            if result:
                results.append(result)

# Save results to CSV
fine_tuning_results = [r['fine_tuning'] for r in results]
forecasting_results = [r['forecasting'] for r in results]

pd.DataFrame(fine_tuning_results).to_csv('/content/fine_tuning_results_svr.csv', index=False)
pd.DataFrame(forecasting_results).to_csv('/content/forecasted_values_svr.csv', index=False)
print("Results saved!")

Results saved!


including aggregate evlaution metric for each district for the above code

---
svr1_district_aggregates_svr.csv



In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pandas as pd
import numpy as np

# Load the dataset
file_path = '/content/Crop_Data_Final.csv'
data = pd.read_csv(file_path)

# Future years to forecast
future_years = [2025, 2030, 2035, 2040, 2045, 2050]

# Crop-related target columns
crop_columns = {
    'Rice': ['RICE AREA (1000 ha)', 'RICE PRODUCTION (1000 tons)', 'RICE YIELD (Kg per ha)'],
    'Wheat': ['WHEAT AREA (1000 ha)', 'WHEAT PRODUCTION (1000 tons)', 'WHEAT YIELD (Kg per ha)'],
    # Add other crops as in the original script...
}

# Hyperparameter tuning grid for SVR
param_distributions = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.1],
    'epsilon': [0.1, 0.01]
}

# Function to calculate metrics
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return mse, r2, mae, rmse

# Preprocessing function for scaling
def preprocess_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, scaler

# Function to process crops for a district
def process_crop(data, crop, target_column):
    X = data.drop(columns=['Year', 'Dist Name'] + crop_columns[crop], errors='ignore')
    y = data[target_column]

    if len(y) < 5:
        return None  # Skip if insufficient data

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Preprocess the data
    X_train_scaled, X_test_scaled, scaler = preprocess_data(X_train, X_test)

    # Hyperparameter tuning
    svr = SVR()
    random_search = RandomizedSearchCV(
        estimator=svr,
        param_distributions=param_distributions,
        n_iter=10,
        cv=2,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )
    random_search.fit(X_train_scaled, y_train)

    # Get the best model
    best_model = random_search.best_estimator_
    best_params = random_search.best_params_

    # Evaluate on test data
    y_pred = best_model.predict(X_test_scaled)
    mse, r2, mae, rmse = calculate_metrics(y_test, y_pred)

    # Prepare future data for forecasting
    future_data = data.iloc[:len(future_years)].copy()
    future_data['Year'] = future_years
    future_X = future_data.drop(columns=['Year', 'Dist Name'] + crop_columns[crop], errors='ignore')
    future_X_scaled = scaler.transform(future_X)
    future_predictions = best_model.predict(future_X_scaled)

    return {
        'fine_tuning': {
            'Crop': crop,
            'Target Column': target_column,
            'Best Parameters': best_params,
            'MSE': mse,
            'R²': r2,
            'MAE': mae,
            'RMSE': rmse
        },
        'forecasting': {
            'Future Years': future_years,
            'Predictions': future_predictions.tolist()
        }
    }

# Process each crop and district
districts = data['Dist Name'].unique()
results = []

district_aggregates = []

for district in districts:
    district_data = data[data['Dist Name'] == district]
    district_metrics = []

    for crop, columns in crop_columns.items():
        for target_column in columns:
            result = process_crop(district_data, crop, target_column)
            if result:
                results.append(result)
                district_metrics.append(result['fine_tuning'])

    # Calculate aggregate metrics for the district
    if district_metrics:
        mse_avg = np.mean([m['MSE'] for m in district_metrics])
        r2_avg = np.mean([m['R²'] for m in district_metrics])
        mae_avg = np.mean([m['MAE'] for m in district_metrics])
        rmse_avg = np.mean([m['RMSE'] for m in district_metrics])

        district_aggregates.append({
            'District': district,
            'Mean MSE': mse_avg,
            'Mean R²': r2_avg,
            'Mean MAE': mae_avg,
            'Mean RMSE': rmse_avg
        })

# Save results to CSV
fine_tuning_results = [r['fine_tuning'] for r in results]
forecasting_results = [r['forecasting'] for r in results]

pd.DataFrame(fine_tuning_results).to_csv('/content/fine_tuning_results_svr.csv', index=False)
pd.DataFrame(forecasting_results).to_csv('/content/forecasted_values_svr.csv', index=False)
pd.DataFrame(district_aggregates).to_csv('/content/district_aggregates_svr.csv', index=False)

print("Results saved!")


Results saved!


running fast

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pandas as pd
import numpy as np

# Load the dataset
file_path = '/content/Crop_Data_Final.csv'
data = pd.read_csv(file_path)

# Future years to forecast
future_years = [2025, 2030, 2035, 2040, 2045, 2050]

# Crop-related target columns
crop_columns = {
    'Rice': ['RICE AREA (1000 ha)', 'RICE PRODUCTION (1000 tons)', 'RICE YIELD (Kg per ha)'],
    'Wheat': ['WHEAT AREA (1000 ha)', 'WHEAT PRODUCTION (1000 tons)', 'WHEAT YIELD (Kg per ha)'],
    'Sorghum': ['SORGHUM AREA (1000 ha)', 'SORGHUM PRODUCTION (1000 tons)', 'SORGHUM YIELD (Kg per ha)'],
    'Pearl Millet': ['PEARL MILLET AREA (1000 ha)', 'PEARL MILLET PRODUCTION (1000 tons)', 'PEARL MILLET YIELD (Kg per ha)'],
    'Maize': ['MAIZE AREA (1000 ha)', 'MAIZE PRODUCTION (1000 tons)', 'MAIZE YIELD (Kg per ha)'],
    'Chickpea': ['CHICKPEA AREA (1000 ha)', 'CHICKPEA PRODUCTION (1000 tons)', 'CHICKPEA YIELD (Kg per ha)'],
    'Pigeonpea': ['PIGEONPEA AREA (1000 ha)', 'PIGEONPEA PRODUCTION (1000 tons)', 'PIGEONPEA YIELD (Kg per ha)'],
    'Minor Pulses': ['MINOR PULSES AREA (1000 ha)', 'MINOR PULSES PRODUCTION (1000 tons)', 'MINOR PULSES YIELD (Kg per ha)'],
    'Groundnut': ['GROUNDNUT AREA (1000 ha)', 'GROUNDNUT PRODUCTION (1000 tons)', 'GROUNDNUT YIELD (Kg per ha)'],
    'Sesamum': ['SESAMUM AREA (1000 ha)', 'SESAMUM PRODUCTION (1000 tons)', 'SESAMUM YIELD (Kg per ha)'],
    'Oilseeds': ['OILSEEDS AREA (1000 ha)', 'OILSEEDS PRODUCTION (1000 tons)', 'OILSEEDS YIELD (Kg per ha)'],
    'Sugarcane': ['SUGARCANE AREA (1000 ha)', 'SUGARCANE PRODUCTION (1000 tons)', 'SUGARCANE YIELD (Kg per ha)'],
    'Cotton': ['COTTON AREA (1000 ha)', 'COTTON PRODUCTION (1000 tons)', 'COTTON YIELD (Kg per ha)'],
    'Fruits and Vegetables': ['FRUITS AND VEGETABLES AREA (1000 ha)'],
    'Fertilizers': ['NITROGEN SHARE IN NPK (Percent)', 'PHOSPHATE SHARE IN NPK (Percent)', 'POTASH SHARE IN NPK (Percent)'],
    'Soil Nutrients': ['NITROGEN PER HA OF NCA (Kg per ha)', 'NITROGEN PER HA OF GCA (Kg per ha)',
                       'PHOSPHATE PER HA OF NCA (Kg per ha)', 'PHOSPHATE PER HA OF GCA (Kg per ha)',
                       'POTASH PER HA OF NCA (Kg per ha)', 'POTASH PER HA OF GCA (Kg per ha)'],
    'Weather': ['Min Temp (Centigrate)', 'Max Temp (Centigrate)', 'Precipitation (mm)', 'Irrigated Area', 'Annual Rainfall']
}

# Hyperparameter tuning grid for SVR
param_distributions = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.1],
    'epsilon': [0.1, 0.01]
}

# Function to calculate metrics
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    r2 = max(0, r2_score(y_true, y_pred))  # Ensure R² is non-negative
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return mse, r2, mae, rmse

# Preprocessing function for scaling
def preprocess_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, scaler

# Function to process crops for a district
def process_crop(data, crop, target_column, district):
    print(f"Processing district: {district}, crop: {crop}, target: {target_column}")
    X = data.drop(columns=['Year', 'Dist Name'] + crop_columns[crop], errors='ignore')
    y = data[target_column]

    if len(y) < 5:
        print(f"Insufficient data for {district}, {crop}, {target_column}. Skipping.")
        return None  # Skip if insufficient data

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Preprocess the data
    X_train_scaled, X_test_scaled, scaler = preprocess_data(X_train, X_test)

    # Hyperparameter tuning
    svr = SVR()
    random_search = RandomizedSearchCV(
        estimator=svr,
        param_distributions=param_distributions,
        n_iter=10,
        cv=2,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )
    random_search.fit(X_train_scaled, y_train)

    # Get the best model
    best_model = random_search.best_estimator_
    best_params = random_search.best_params_

    # Evaluate on test data
    y_pred = best_model.predict(X_test_scaled)
    mse, r2, mae, rmse = calculate_metrics(y_test, y_pred)

    # Log R² status
    if r2 == 0:
        print(f"Warning: Poor performance (R² = {r2}) for {district}, {crop}, {target_column}")

    # Prepare future data for forecasting
    future_data = data.iloc[:len(future_years)].copy()
    future_data['Year'] = future_years
    future_X = future_data.drop(columns=['Year', 'Dist Name'] + crop_columns[crop], errors='ignore')
    future_X_scaled = scaler.transform(future_X)
    future_predictions = best_model.predict(future_X_scaled)

    return {
        'fine_tuning': {
            'District': district,
            'Crop': crop,
            'Target Column': target_column,
            'Best Parameters': best_params,
            'MSE': mse,
            'R²': r2,
            'MAE': mae,
            'RMSE': rmse
        },
        'forecasting': {
            'District': district,
            'Crop': crop,
            'Future Years': future_years,
            'Predictions': future_predictions.tolist()
        }
    }

# Process each crop and district
districts = data['Dist Name'].unique()
results = []

for district in districts:
    district_data = data[data['Dist Name'] == district]
    for crop, columns in crop_columns.items():
        for target_column in columns:
            result = process_crop(district_data, crop, target_column, district)
            if result:
                results.append(result)

# Save results to CSV
fine_tuning_results = [r['fine_tuning'] for r in results]
forecasting_results = [r['forecasting'] for r in results]

pd.DataFrame(fine_tuning_results).to_csv('/content/fine_tuning_results_svr.csv', index=False)
pd.DataFrame(forecasting_results).to_csv('/content/forecasted_values_svr.csv', index=False)
print("Results saved!")


Processing district: Ahmednagar, crop: Rice, target: RICE AREA (1000 ha)
Processing district: Ahmednagar, crop: Rice, target: RICE PRODUCTION (1000 tons)
Processing district: Ahmednagar, crop: Rice, target: RICE YIELD (Kg per ha)
Processing district: Ahmednagar, crop: Wheat, target: WHEAT AREA (1000 ha)
Processing district: Ahmednagar, crop: Wheat, target: WHEAT PRODUCTION (1000 tons)
Processing district: Ahmednagar, crop: Wheat, target: WHEAT YIELD (Kg per ha)
Processing district: Ahmednagar, crop: Sorghum, target: SORGHUM AREA (1000 ha)
Processing district: Ahmednagar, crop: Sorghum, target: SORGHUM PRODUCTION (1000 tons)
Processing district: Ahmednagar, crop: Sorghum, target: SORGHUM YIELD (Kg per ha)
Processing district: Ahmednagar, crop: Pearl Millet, target: PEARL MILLET AREA (1000 ha)
Processing district: Ahmednagar, crop: Pearl Millet, target: PEARL MILLET PRODUCTION (1000 tons)
Processing district: Ahmednagar, crop: Pearl Millet, target: PEARL MILLET YIELD (Kg per ha)
Processi

including aggregate fir the above running fast

---
aggregate_evaluation_metrics_svr.csv


In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pandas as pd
import numpy as np

# Load the dataset
file_path = '/content/Crop_Data_Final.csv'
data = pd.read_csv(file_path)

# Future years to forecast
future_years = [2025, 2030, 2035, 2040, 2045, 2050]

# Crop-related target columns
crop_columns = {
    'Rice': ['RICE AREA (1000 ha)', 'RICE PRODUCTION (1000 tons)', 'RICE YIELD (Kg per ha)'],
    'Wheat': ['WHEAT AREA (1000 ha)', 'WHEAT PRODUCTION (1000 tons)', 'WHEAT YIELD (Kg per ha)'],
    'Sorghum': ['SORGHUM AREA (1000 ha)', 'SORGHUM PRODUCTION (1000 tons)', 'SORGHUM YIELD (Kg per ha)'],
    'Pearl Millet': ['PEARL MILLET AREA (1000 ha)', 'PEARL MILLET PRODUCTION (1000 tons)', 'PEARL MILLET YIELD (Kg per ha)'],
    'Maize': ['MAIZE AREA (1000 ha)', 'MAIZE PRODUCTION (1000 tons)', 'MAIZE YIELD (Kg per ha)'],
    'Chickpea': ['CHICKPEA AREA (1000 ha)', 'CHICKPEA PRODUCTION (1000 tons)', 'CHICKPEA YIELD (Kg per ha)'],
    'Pigeonpea': ['PIGEONPEA AREA (1000 ha)', 'PIGEONPEA PRODUCTION (1000 tons)', 'PIGEONPEA YIELD (Kg per ha)'],
    'Minor Pulses': ['MINOR PULSES AREA (1000 ha)', 'MINOR PULSES PRODUCTION (1000 tons)', 'MINOR PULSES YIELD (Kg per ha)'],
    'Groundnut': ['GROUNDNUT AREA (1000 ha)', 'GROUNDNUT PRODUCTION (1000 tons)', 'GROUNDNUT YIELD (Kg per ha)'],
    'Sesamum': ['SESAMUM AREA (1000 ha)', 'SESAMUM PRODUCTION (1000 tons)', 'SESAMUM YIELD (Kg per ha)'],
    'Oilseeds': ['OILSEEDS AREA (1000 ha)', 'OILSEEDS PRODUCTION (1000 tons)', 'OILSEEDS YIELD (Kg per ha)'],
    'Sugarcane': ['SUGARCANE AREA (1000 ha)', 'SUGARCANE PRODUCTION (1000 tons)', 'SUGARCANE YIELD (Kg per ha)'],
    'Cotton': ['COTTON AREA (1000 ha)', 'COTTON PRODUCTION (1000 tons)', 'COTTON YIELD (Kg per ha)'],
    'Fruits and Vegetables': ['FRUITS AND VEGETABLES AREA (1000 ha)'],
    'Fertilizers': ['NITROGEN SHARE IN NPK (Percent)', 'PHOSPHATE SHARE IN NPK (Percent)', 'POTASH SHARE IN NPK (Percent)'],
    'Soil Nutrients': ['NITROGEN PER HA OF NCA (Kg per ha)', 'NITROGEN PER HA OF GCA (Kg per ha)',
                       'PHOSPHATE PER HA OF NCA (Kg per ha)', 'PHOSPHATE PER HA OF GCA (Kg per ha)',
                       'POTASH PER HA OF NCA (Kg per ha)', 'POTASH PER HA OF GCA (Kg per ha)'],
    'Weather': ['Min Temp (Centigrate)', 'Max Temp (Centigrate)', 'Precipitation (mm)', 'Irrigated Area', 'Annual Rainfall']
}

# Hyperparameter tuning grid for SVR
param_distributions = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.1],
    'epsilon': [0.1, 0.01]
}

# Function to calculate metrics
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    r2 = max(0, r2_score(y_true, y_pred))  # Ensure R² is non-negative
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return mse, r2, mae, rmse

# Preprocessing function for scaling
def preprocess_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, scaler

# Function to process crops for a district
def process_crop(data, crop, target_column, district):
    print(f"Processing district: {district}, crop: {crop}, target: {target_column}")
    X = data.drop(columns=['Year', 'Dist Name'] + crop_columns[crop], errors='ignore')
    y = data[target_column]

    if len(y) < 5:
        print(f"Insufficient data for {district}, {crop}, {target_column}. Skipping.")
        return None  # Skip if insufficient data

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Preprocess the data
    X_train_scaled, X_test_scaled, scaler = preprocess_data(X_train, X_test)

    # Hyperparameter tuning
    svr = SVR()
    random_search = RandomizedSearchCV(
        estimator=svr,
        param_distributions=param_distributions,
        n_iter=10,
        cv=2,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )
    random_search.fit(X_train_scaled, y_train)

    # Get the best model
    best_model = random_search.best_estimator_
    best_params = random_search.best_params_

    # Evaluate on test data
    y_pred = best_model.predict(X_test_scaled)
    mse, r2, mae, rmse = calculate_metrics(y_test, y_pred)

    # Log R² status
    if r2 == 0:
        print(f"Warning: Poor performance (R² = {r2}) for {district}, {crop}, {target_column}")

    # Prepare future data for forecasting
    future_data = data.iloc[:len(future_years)].copy()
    future_data['Year'] = future_years
    future_X = future_data.drop(columns=['Year', 'Dist Name'] + crop_columns[crop], errors='ignore')
    future_X_scaled = scaler.transform(future_X)
    future_predictions = best_model.predict(future_X_scaled)

    return {
        'fine_tuning': {
            'District': district,
            'Crop': crop,
            'Target Column': target_column,
            'Best Parameters': best_params,
            'MSE': mse,
            'R²': r2,
            'MAE': mae,
            'RMSE': rmse
        },
        'forecasting': {
            'District': district,
            'Crop': crop,
            'Future Years': future_years,
            'Predictions': future_predictions.tolist()
        }
    }

# Process each crop and district
districts = data['Dist Name'].unique()
results = []

for district in districts:
    district_data = data[data['Dist Name'] == district]

    # Initialize aggregates for the district
    district_metrics = {'District': district, 'MSE': [], 'R²': [], 'MAE': [], 'RMSE': []}
    district_forecasts = {'District': district, 'Crop': [], 'Future Years': future_years, 'Predictions': []}

    for crop, columns in crop_columns.items():
        for target_column in columns:
            result = process_crop(district_data, crop, target_column, district)
            if result:
                results.append(result)
                # Aggregate metrics for the district
                district_metrics['MSE'].append(result['fine_tuning']['MSE'])
                district_metrics['R²'].append(result['fine_tuning']['R²'])
                district_metrics['MAE'].append(result['fine_tuning']['MAE'])
                district_metrics['RMSE'].append(result['fine_tuning']['RMSE'])

                # Aggregate forecasting results
                district_forecasts['Crop'].append(f"{crop} - {target_column}")
                district_forecasts['Predictions'].append(result['forecasting']['Predictions'])

    # Aggregate district-level metrics
    district_metrics['MSE'] = np.mean(district_metrics['MSE'])
    district_metrics['R²'] = np.mean(district_metrics['R²'])
    district_metrics['MAE'] = np.mean(district_metrics['MAE'])
    district_metrics['RMSE'] = np.mean(district_metrics['RMSE'])

    # Save aggregated district results
    results.append({'fine_tuning': district_metrics, 'forecasting': district_forecasts})

# Save results to CSV
fine_tuning_results = [r['fine_tuning'] for r in results]
forecasting_results = [r['forecasting'] for r in results]

pd.DataFrame(fine_tuning_results).to_csv('/content/fine_tuning_results_svr.csv', index=False)
pd.DataFrame(forecasting_results).to_csv('/content/forecasted_values_svr.csv', index=False)
print("Results saved!")

# Aggregate evaluation metrics for each district
district_metrics_results = []

for district in districts:
    district_metrics_results.append({
        'District': district,
        'MSE': district_metrics['MSE'],
        'R²': district_metrics['R²'],
        'MAE': district_metrics['MAE'],
        'RMSE': district_metrics['RMSE']
    })

# Save the aggregated metrics to a separate CSV
pd.DataFrame(district_metrics_results).to_csv('/content/aggregate_evaluation_metrics_svr.csv', index=False)

print("Aggregate evaluation metrics saved!")


Processing district: Ahmednagar, crop: Rice, target: RICE AREA (1000 ha)
Processing district: Ahmednagar, crop: Rice, target: RICE PRODUCTION (1000 tons)
Processing district: Ahmednagar, crop: Rice, target: RICE YIELD (Kg per ha)
Processing district: Ahmednagar, crop: Wheat, target: WHEAT AREA (1000 ha)
Processing district: Ahmednagar, crop: Wheat, target: WHEAT PRODUCTION (1000 tons)
Processing district: Ahmednagar, crop: Wheat, target: WHEAT YIELD (Kg per ha)
Processing district: Ahmednagar, crop: Sorghum, target: SORGHUM AREA (1000 ha)
Processing district: Ahmednagar, crop: Sorghum, target: SORGHUM PRODUCTION (1000 tons)
Processing district: Ahmednagar, crop: Sorghum, target: SORGHUM YIELD (Kg per ha)
Processing district: Ahmednagar, crop: Pearl Millet, target: PEARL MILLET AREA (1000 ha)
Processing district: Ahmednagar, crop: Pearl Millet, target: PEARL MILLET PRODUCTION (1000 tons)
Processing district: Ahmednagar, crop: Pearl Millet, target: PEARL MILLET YIELD (Kg per ha)
Processi